In [8]:
import os
import json
from collections import Counter, defaultdict

# Ruta base donde buscar los archivos .json
base_dir = "../datasets/fallos_json_modificados"

# Diccionario para almacenar la información
result = {}

# Recorrer todas las carpetas y archivos .json
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                citations = (
                    data.get("METADATOS", {})
                    .get("ARTICULOS_CITADOS", {})
                    .get("citations", [])
                )
                for citation in citations:
                    main_source = citation.get("main_source")
                    cited_articles = citation.get("cited_articles", [])
                    if main_source is not None and cited_articles is not None:
                        if main_source not in result:
                            result[main_source] = []
                        result[main_source].extend(cited_articles)
            except Exception as e:
                print(f"Error en {file_path}: {e}")

# Construir el JSON de salida
output = {}
for source, articles in result.items():
    freq = Counter(articles)
    output[source] = {
        "cited_articles_frequency": dict(freq),
        "total_cited_articles": sum(freq.values())
    }

# Guardar el resultado en un archivo
with open("main_source_cited_articles_stats_modificado.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print("Archivo 'main_source_cited_articles_stats.json' generado correctamente.")

Archivo 'main_source_cited_articles_stats.json' generado correctamente.


In [9]:
import json

# Abrir el archivo generado
with open("main_source_cited_articles_stats_modificado.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear un diccionario con main_source y su frecuencia total
total_freq = {k: v["total_cited_articles"] for k, v in data.items()}

# Mostrar el resultado como JSON puro
print(json.dumps(total_freq, ensure_ascii=False, indent=2))


{
  "Ley 7046": 723,
  "Acuerdo General 15/18 SNE": 505,
  "Código Civil y Comercial (CCC)": 414,
  "Ley 19.550": 3,
  "Código Procesal Civil y Comercial (CPCC)": 718,
  "Acuerdo General 11/20": 32,
  "Ley 24.449": 54,
  "Corte Suprema de Justicia de la Nación (CSJN)": 74,
  "Constitución Nacional (CN)": 144,
  "Otro": 180,
  "Ley 10.636": 1,
  "Ley 5796": 1,
  "Código Procesal Civil y Comercial de Entre Ríos (CPCCER)": 15,
  "Ley 48": 15,
  "Acordada 4/2007": 47,
  "Ley 25.529": 6,
  "Ley 4878": 8,
  "Ley 27.423": 2,
  "Ley 9776": 1,
  "Código Procesal Penal (CPP)": 23,
  "Ley 23.737": 6,
  "Acuerdo General 28/20": 1,
  "Ley 6902": 4,
  "Constitución Provincial de Entre Ríos": 25,
  "Ley 10.704": 179,
  "Reglamento de Presentaciones Electrónicas": 6,
  "Ley 10.668": 2,
  "Ley 26.485": 5,
  "Ley 10.855": 2,
  "Ley 9861": 4,
  "Ley 26.061": 9,
  "Resolución 170/19": 6,
  "Ley 24.522 (LCQ - Ley de Concursos y Quiebras)": 28,
  "Convención Americana de Derechos Humanos (Pacto de San José 

In [2]:
import json
import re
from pathlib import Path

def clean_idea_central(json_path):
    """
    Limpia el campo IDEA_CENTRAL eliminando encabezados redundantes
    
    Args:
        json_path (str): Ruta al archivo JSON a procesar
    """
    # Leer el archivo JSON
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    # Verificar si existe el campo IDEA_CENTRAL
    if "IDEA_CENTRAL" in data:
        idea_central = data["IDEA_CENTRAL"]
        
        # Patrones a eliminar (con variaciones)
        patterns_to_remove = [
            r"^\*\*IDEA CENTRAL DEL FALLO\*\*\n\n",
            r"^### IDEA CENTRAL DEL FALLO\n\n",
            r"^\*\*IDEA CENTRAL DEL FALLO:\*\*\n\n",
            r"^### IDEA CENTRAL DEL FALLO:\n\n",
            r"^\*\*IDEA CENTRAL:\*\*\n\n",
            r"^### IDEA CENTRAL:\n\n",
            r"^\*\*IDEA CENTRAL DEL FALLO\*\*\n",
            r"^### IDEA CENTRAL DEL FALLO\n",
            # Nuevos patrones corregidos
            r"^\*\*IDEA CENTRAL DEL FALLO JUDICIAL\*\*\n\n",
            r"^\*\*IDEA CENTRAL DEL FALLO JUDICIAL\*\*\n",
            r"^\*\*Decisión Judicial Principal:\*\*\n\n",
            r"^\*\*Decisión Judicial Principal:\*\*\n",
            r"^\*\*Decisión Judicial Principal:\*\*",
            r"^\*\*decisión judicial principal:\*\*\n",
            # Patrones con espacios al inicio
            r"^\s+\*\*IDEA CENTRAL DEL FALLO\*\*\n\n",
            r"^\s+### IDEA CENTRAL DEL FALLO\n\n",
            r"^\s+\*\*IDEA CENTRAL DEL FALLO JUDICIAL\*\*\n\n",
            r"^\s+\*\*Decisión Judicial Principal:\*\*\n\n",
            r"^\s+\*\*Decisión Judicial Principal:\*\*\n",
            r"^\s+\*\*Decisión Judicial Principal:\*\*",
            r"^\s+\*\*decisión judicial principal:\*\*",
        ]
        
        # Aplicar cada patrón
        cleaned_text = idea_central
        for pattern in patterns_to_remove:
            cleaned_text = re.sub(pattern, "", cleaned_text, flags=re.MULTILINE)
        
        # Actualizar el JSON solo si hubo cambios
        if cleaned_text != idea_central:
            data["IDEA_CENTRAL"] = cleaned_text
            print(f"✓ Limpiado encabezado en: {json_path}")
            
            # Guardar el archivo modificado
            with open(json_path, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=2)
            return True
        else:
            print(f"- Sin cambios en: {json_path}")
            return False
    else:
        print(f"⚠ No se encontró campo IDEA_CENTRAL en: {json_path}")
        return False

def main():
    # Directorio base
    base_dir = Path("../datasets/fallos_json_modificados")
    
    # Verificar que el directorio existe
    if not base_dir.exists():
        print(f"❌ El directorio no existe: {base_dir}")
        return
    
    # Buscar todos los archivos JSON recursivamente
    json_files = list(base_dir.rglob("*.json"))
    
    if not json_files:
        print(f"❌ No se encontraron archivos JSON en: {base_dir}")
        return
    
    print(f"📁 Encontrados {len(json_files)} archivos JSON")
    print("🚀 Iniciando procesamiento...\n")
    
    # Contadores y listas
    processed = 0
    modified = 0
    errors = 0
    not_modified = []
    
    # Procesar cada archivo
    for json_file in json_files:
        try:
            if clean_idea_central(json_file):
                modified += 1
            else:
                not_modified.append(json_file)
            processed += 1
        except Exception as e:
            print(f"❌ Error procesando {json_file}: {e}")
            errors += 1
    
    # Resumen final
    print(f"\n📊 RESUMEN:")
    print(f"   Archivos procesados: {processed}")
    print(f"   Archivos modificados: {modified}")
    print(f"   Archivos sin modificar: {len(not_modified)}")
    print(f"   Errores: {errors}")
    
    if not_modified:
        print(f"\n📋 ARCHIVOS SIN MODIFICAR:")
        for file in not_modified:
            print(f"   - {file}")
    
    print(f"\n🎉 Proceso completado!")

if __name__ == "__main__":
    main()

📁 Encontrados 295 archivos JSON
🚀 Iniciando procesamiento...

✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8104.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8142.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8344.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8569.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8752.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8865.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8926.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8927.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8948.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\02\8971.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modificados\2024\03\8613.json
✓ Limpiado encabezado en: ..\datasets\fallos_json_modifica

In [ ]:
import os
import json
from collections import defaultdict

# Ruta base donde buscar los archivos .json
base_dir = "../datasets/fallos_json"
# Nueva carpeta de salida
output_base_dir = "../datasets/fallos_json_modificados"

# Artículos a mover de 'Otro' a cada ley
articulos_ley_7046 = {28, 29, 114}
articulos_ley_10704 = {33}

# Recorrer todas las carpetas y archivos .json
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                citations = (
                    data.get("METADATOS", {})
                    .get("ARTICULOS_CITADOS", {})
                    .get("citations", [])
                )
                # Agrupar main_source únicos y eliminar duplicados
                grouped = defaultdict(lambda: {"cited_articles": set(), "extra": set()})
                for citation in citations:
                    main_source = citation.get("main_source")
                    cited_articles = citation.get("cited_articles")
                    extra = citation.get("extra")
                    if extra is None and "extras" in citation:
                        extra = citation.get("extras")
                    if cited_articles is None:
                        cited_articles = []
                    if extra is None:
                        extra = []
                    if isinstance(extra, str):
                        extra = [extra]
                    if main_source is not None:
                        grouped[main_source]["cited_articles"].update(cited_articles)
                        grouped[main_source]["extra"].update(extra)
                # --- MODIFICACIÓN: Mover artículos de 'Otro' a las leyes correspondientes ---
                otros = grouped.get("Otro")
                if otros:
                    articulos_otro = otros["cited_articles"]
                    # Ley 7046
                    articulos_mover_7046 = articulos_otro & articulos_ley_7046
                    if articulos_mover_7046:
                        if "Ley 7046" not in grouped:
                            grouped["Ley 7046"] = {"cited_articles": set(), "extra": set()}
                        grouped["Ley 7046"]["cited_articles"].update(articulos_mover_7046)
                        articulos_otro.difference_update(articulos_mover_7046)
                    # Ley 10.704
                    articulos_mover_10704 = articulos_otro & articulos_ley_10704
                    if articulos_mover_10704:
                        if "Ley 10.704" not in grouped:
                            grouped["Ley 10.704"] = {"cited_articles": set(), "extra": set()}
                        grouped["Ley 10.704"]["cited_articles"].update(articulos_mover_10704)
                        articulos_otro.difference_update(articulos_mover_10704)
                # Eliminar 'Otro' si queda vacío
                if "Otro" in grouped and not grouped["Otro"]["cited_articles"]:
                    del grouped["Otro"]
                # Reconstruir la lista de citas agrupadas y sin duplicados
                new_citations = []
                for main_source, vals in grouped.items():
                    new_citations.append({
                        "main_source": main_source,
                        "cited_articles": sorted(list(vals["cited_articles"])),
                        "extra": list(vals["extra"])
                    })
                # Guardar los cambios en la nueva carpeta, replicando la estructura
                if "METADATOS" in data and "ARTICULOS_CITADOS" in data["METADATOS"]:
                    data["METADATOS"]["ARTICULOS_CITADOS"]["citations"] = new_citations
                    # Construir ruta de salida
                    rel_path = os.path.relpath(file_path, base_dir)
                    output_path = os.path.join(output_base_dir, rel_path)
                    os.makedirs(os.path.dirname(output_path), exist_ok=True)
                    with open(output_path, "w", encoding="utf-8") as f:
                        json.dump(data, f, ensure_ascii=False, indent=2)
            except Exception as e:
                print(f"Error en {file_path}: {e}")

print("Archivos .json modificados guardados en la carpeta 'fallos_json_modificados', replicando la estructura original.")

In [ ]:
import json

# Abrir el archivo generado
with open("main_source_cited_articles_stats_modificado.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear un diccionario con main_source y su frecuencia total
total_freq = {k: v["total_cited_articles"] for k, v in data.items()}

# Mostrar el resultado como JSON
import pprint
pprint.pprint(total_freq)
